In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import yaml
import random

Контракты могут быть трех типов:

1) __Потоки от backup трафика__ - большой объем передаваемых данных, большой интервал времени\
2) __Миграция виртуальных машин__ - большой объем трафика (но меньший чем в п.1), маленький интервал времени\
3) __Стандартный трафик между виртуальными машинами__ (взаимодействие между vm в рамках одного сервиса) - объем данных маленький, время передачи маленькое

In [20]:
TOPO_NAME = 'cBren'
CTYPE1 = 10
CTYPE2 = 10
CTYPE3 = 10
UCON_NUM = CTYPE1 + CTYPE2 + CTYPE3
ACON_NUM = 70
T_MAX = 2000
PATH = 'Unique_Contracts/{}'.format(TOPO_NAME)

In [21]:
H1 = nx.read_graphml('topo/{}.graphml'.format(TOPO_NAME))
H = nx.Graph(H1)
H

In [22]:
Paths_len = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))

In [23]:
for i in nx.shortest_path_length(H):
    for k in i[1].keys():
        Paths_len[int(i[0])][int(k)] = i[1][k]

In [24]:
# Paths_len

In [25]:
Paths_len.shape

(37, 37)

In [26]:
Lenghts = {}
for i in range(Paths_len.shape[0]):
    for j in range(Paths_len.shape[1]):
        k = Paths_len[i][j]
        if not k in Lenghts.keys():
            Lenghts[k] = [(i, j)]
        elif not ((i, j) in Lenghts[k] or (j, i) in Lenghts[k]):
            Lenghts[k] += [(i, j)]

In [27]:
# Lenghts

In [28]:
vertex = []
for i in range(UCON_NUM):
    keys = Lenghts.keys()
    m = max(keys)
    vertex += [Lenghts[m][0]]
    del Lenghts[m][0]
    if Lenghts[m] == []:
        del Lenghts[m]

In [29]:
len(vertex)

30

In [30]:
def factor(n):
    Ans = []
    d = 2
    while d * d <= n:
        if n % d == 0:
            Ans.append(d)
            n //= d
        else:
            d += 1
    if n > 1:
        Ans.append(n)
    return Ans

def get_CTYPE1():
    # 1GB - 3GB
    v = random.randrange(2*(10**3), 3*(10**3), 10)
    f = factor(v)
    # 8 min - 12.5 min
    t = int(np.prod(f[2:]))
    return (v, t)
    
def get_CTYPE2():
    # 300MB - 1GB
    v = random.randrange(300, 1000, 10)
    f = factor(v)
    # 8 min - 12.5 min
    t = int(np.prod(f[2:]))
    return (v, t)
    
def get_CTYPE3():
    # 10MB - 200MB
    v = random.randrange(10, 200, 10)
    f = factor(v)
    # 8 min - 12.5 min
    t = int(np.prod(f[2:]))
    return (v, t)
    
def init_CTYPE1(contr):
    for i in range(CTYPE1):
        v, t = get_CTYPE1()
        a, b = random.choice(vertex)
        vertex.remove((a, b))
        contr += [{'volume': v,
                      't_s': [],
                       't': t,
                       'A': a,
                       'B': b,
                       'type': 1}]
    
def init_CTYPE2(contr):
    for i in range(CTYPE2):
        v, t = get_CTYPE2()
        a, b = random.choice(vertex)
        vertex.remove((a, b))
        contr += [{'volume': v,
                      't_s': [],
                      't': t,
                      'A': a,
                      'B': b,
                      'type': 2}]
    
def init_CTYPE3(contr):
    for i in range(CTYPE3):
        v, t = get_CTYPE3()
        a, b = random.choice(vertex)
        vertex.remove((a, b))
        contr += [{'volume': v,
                      't_s': [],
                      't': t,
                      'A': a,
                      'B': b,
                      'type': 3}]

In [31]:
f = factor(2000)
    # 3 min - 12.5 min
print(f)
np.prod(f[2:])

[2, 2, 2, 2, 5, 5, 5]


500

In [32]:
Unique_contracts = []
init_CTYPE1(Unique_contracts)
init_CTYPE2(Unique_contracts)
init_CTYPE3(Unique_contracts)
Unique_contracts

[{'volume': 2220, 't_s': [], 't': 555, 'A': 5, 'B': 16, 'type': 1},
 {'volume': 2550, 't_s': [], 't': 425, 'A': 18, 'B': 31, 'type': 1},
 {'volume': 2700, 't_s': [], 't': 675, 'A': 18, 'B': 33, 'type': 1},
 {'volume': 2020, 't_s': [], 't': 505, 'A': 16, 'B': 35, 'type': 1},
 {'volume': 2950, 't_s': [], 't': 295, 'A': 4, 'B': 16, 'type': 1},
 {'volume': 2630, 't_s': [], 't': 263, 'A': 24, 'B': 36, 'type': 1},
 {'volume': 2720, 't_s': [], 't': 680, 'A': 5, 'B': 18, 'type': 1},
 {'volume': 2900, 't_s': [], 't': 725, 'A': 18, 'B': 32, 'type': 1},
 {'volume': 2050, 't_s': [], 't': 205, 'A': 6, 'B': 16, 'type': 1},
 {'volume': 2170, 't_s': [], 't': 217, 'A': 18, 'B': 27, 'type': 1},
 {'volume': 400, 't_s': [], 't': 100, 'A': 16, 'B': 36, 'type': 2},
 {'volume': 910, 't_s': [], 't': 91, 'A': 18, 'B': 35, 'type': 2},
 {'volume': 760, 't_s': [], 't': 190, 'A': 9, 'B': 36, 'type': 2},
 {'volume': 440, 't_s': [], 't': 110, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 610, 't_s': [], 't': 61, 'A': 17

Сохраним уникальные контракты (проверить путь **PATH!**)

In [33]:
if not os.path.exists(PATH):
    os.makedirs(PATH)
with open('{}/C={}_CT1-{}_CT2-{}_CT3-{}.yaml'.format(PATH, UCON_NUM, CTYPE1, CTYPE2, CTYPE3), 'w') as f:
    yaml.dump(Unique_contracts, f)

In [105]:
Contracts = []
for i in range(ACON_NUM):
    t_s = int(random.uniform(0, T_MAX))
    while(True):
        i = random.randint(0, UCON_NUM - 1)
        c = Unique_contracts[i]
        if type(c['t_s']) == int:
            d = [c['t_s']]
        else:
            d = c['t_s']
        if T_MAX - c['t'] < t_s:
            continue
        status = True
        for j in d:
            if t_s >= j and t_s <= j + c['t']:
                status = False
        if not status:
            continue
        Unique_contracts[i]['t_s'] += [t_s]
        c = {'volume': Unique_contracts[i]['volume'],
                't_s': t_s,
                  't': Unique_contracts[i]['t'],
                  'A': Unique_contracts[i]['A'],
                  'B': Unique_contracts[i]['B'],
                  'type': Unique_contracts[i]['type']}
        Contracts += [c]
        break  
Contracts

[{'volume': 30, 't_s': 1988, 't': 5, 'A': 5, 'B': 16, 'type': 3},
 {'volume': 910, 't_s': 1352, 't': 91, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 410, 't_s': 277, 't': 41, 'A': 16, 'B': 31, 'type': 2},
 {'volume': 2370, 't_s': 1590, 't': 395, 'A': 18, 'B': 35, 'type': 1},
 {'volume': 2480, 't_s': 953, 't': 620, 'A': 18, 'B': 27, 'type': 1},
 {'volume': 190, 't_s': 1618, 't': 19, 'A': 23, 'B': 36, 'type': 3},
 {'volume': 150, 't_s': 997, 't': 25, 'A': 5, 'B': 18, 'type': 3},
 {'volume': 650, 't_s': 1711, 't': 65, 'A': 4, 'B': 18, 'type': 2},
 {'volume': 2400, 't_s': 256, 't': 600, 'A': 16, 'B': 33, 'type': 1},
 {'volume': 550, 't_s': 1274, 't': 55, 'A': 18, 'B': 36, 'type': 2},
 {'volume': 910, 't_s': 1775, 't': 91, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 800, 't_s': 1788, 't': 200, 'A': 13, 'B': 36, 'type': 2},
 {'volume': 2600, 't_s': 344, 't': 650, 'A': 7, 'B': 16, 'type': 1},
 {'volume': 910, 't_s': 1258, 't': 91, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 840, 't_s': 786, 't': 21

Сохраним результирующий набор контрактов, с уже определенным временем старта (проверить путь **PATH!**)

In [106]:
if not os.path.exists(PATH):
    os.makedirs(PATH)
with open('{}/C={}_TMAX-{}_CT1-{}_CT2-{}_CT3-{}.yaml'.format(PATH, ACON_NUM, T_MAX, CTYPE1, CTYPE2, CTYPE3), 'w') as f:
    yaml.dump(Contracts, f)
Contracts

[{'volume': 30, 't_s': 1988, 't': 5, 'A': 5, 'B': 16, 'type': 3},
 {'volume': 910, 't_s': 1352, 't': 91, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 410, 't_s': 277, 't': 41, 'A': 16, 'B': 31, 'type': 2},
 {'volume': 2370, 't_s': 1590, 't': 395, 'A': 18, 'B': 35, 'type': 1},
 {'volume': 2480, 't_s': 953, 't': 620, 'A': 18, 'B': 27, 'type': 1},
 {'volume': 190, 't_s': 1618, 't': 19, 'A': 23, 'B': 36, 'type': 3},
 {'volume': 150, 't_s': 997, 't': 25, 'A': 5, 'B': 18, 'type': 3},
 {'volume': 650, 't_s': 1711, 't': 65, 'A': 4, 'B': 18, 'type': 2},
 {'volume': 2400, 't_s': 256, 't': 600, 'A': 16, 'B': 33, 'type': 1},
 {'volume': 550, 't_s': 1274, 't': 55, 'A': 18, 'B': 36, 'type': 2},
 {'volume': 910, 't_s': 1775, 't': 91, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 800, 't_s': 1788, 't': 200, 'A': 13, 'B': 36, 'type': 2},
 {'volume': 2600, 't_s': 344, 't': 650, 'A': 7, 'B': 16, 'type': 1},
 {'volume': 910, 't_s': 1258, 't': 91, 'A': 26, 'B': 36, 'type': 2},
 {'volume': 840, 't_s': 786, 't': 21